In [1]:
!export YOLOX_DATADIR=/efs/data/coco_data/
%env YOLOX_DATADIR=/efs/data/coco_data/


env: YOLOX_DATADIR=/efs/data/coco_data/


In [6]:
!echo $YOLOX_DATADIR

/efs/data/coco_data/


In [2]:
model = "yolox-s"
num_accelerators = 1
total_batch_size = 2

In [3]:

env_var_options = "NEURON_RT_ASYNC_EXEC_MAX_INFLIGHT_REQUESTS=3  " + \
    "NEURON_CC_FLAGS=\'--cache_dir=./compiler_cache --model-type=cnn-training\'"

# Compile the model

In [32]:
%%time
import subprocess
print("Compile model")
COMPILE_CMD = f"""{env_var_options} neuron_parallel_compile python -m \
   yolox.tools.train \
    -n {model} \
    -d {num_accelerators} \
    -b {total_batch_size} \
    """

print(f'Running command: \n{COMPILE_CMD}')
if subprocess.check_call(COMPILE_CMD,shell=True):
   print("There was an error with the compilation command")
else:
   print("Compilation Success!!!")

Compile model
Running command: 
NEURON_RT_ASYNC_EXEC_MAX_INFLIGHT_REQUESTS=3  NEURON_CC_FLAGS='--cache_dir=./compiler_cache --model-type=cnn-training' neuron_parallel_compile python -m    yolox.tools.train     -n yolox-s     -d 1     -b 2     


2024-08-27 22:04:54.000003:  140122  INFO ||NEURON_PARALLEL_COMPILE||: Removing existing workdir /tmp/ubuntu/parallel_compile_workdir
2024-08-27 22:04:54.000004:  140122  INFO ||NEURON_PARALLEL_COMPILE||: Running trial run (add option to terminate trial run early; also ignore trial run's generated outputs, i.e. loss, checkpoints)
2024-08-27 22:04:54.000004:  140122  INFO ||NEURON_PARALLEL_COMPILE||: Running cmd: ['python', '-m', 'yolox.tools.train', '-n', 'yolox-s', '-d', '1', '-b', '2']
/bin/sh: 1: nvidia-smi: not found
2024-08-27 22:05:05 | INFO     | yolox.core.trainer:140 - args: Namespace(experiment_name='yolox_s', name='yolox-s', dist_backend='nccl', dist_url=None, batch_size=2, devices=1, exp_file=None, resume=False, ckpt=None, start_epoch=None, num_machines=1, machine_rank=0, fp16=False, cache=None, occupy=False, logger='tensorboard', opts=[])
2024-08-27 22:05:05 | INFO     | yolox.core.trainer:141 - exp value:
���═���══��══���══���══��══�����═�����═══��══�����══����═�����══���

KeyboardInterrupt: 

......
Compiler status PASS
2024-08-27 22:06:45.000352:  140326  INFO ||NEURON_PARALLEL_COMPILE||: worker 1 finished with num of tasks 1....
2024-08-27 22:06:45.000354:  140326  INFO ||NEURON_CACHE||: Current remaining items are 2, locked are 2, failed are 0, done are 96, total is 100
............
Compiler status PASS
2024-08-27 22:08:33.000001:  140328  INFO ||NEURON_PARALLEL_COMPILE||: worker 3 finished with num of tasks 1....
2024-08-27 22:08:33.000003:  140328  INFO ||NEURON_CACHE||: Current remaining items are 2, locked are 1, failed are 0, done are 97, total is 100

Compiler status PASS
2024-08-27 22:08:41.000771:  140325  INFO ||NEURON_PARALLEL_COMPILE||: worker 0 finished with num of tasks 1....
2024-08-27 22:08:41.000773:  140325  INFO ||NEURON_CACHE||: Current remaining items are 2, locked are 0, failed are 0, done are 98, total is 100
2024-08-27 22:08:41.000786:  140122  INFO ||NEURON_PARALLEL_COMPILE||: {
    "compilation_summary": {
        "true": 4
    },
    "compilatio

In [11]:
%%time
print("Train model")
RUN_CMD = f"""
   {env_var_options} python -m \
   yolox.tools.train \
    -n {model} \
    -d {num_accelerators} \
    -b {total_batch_size} \
    """

print(f'Running command: \n{RUN_CMD}')
if subprocess.check_call(RUN_CMD,shell=True):
   print("There was an error with the fine-tune command")
else:
   print("Fine-tune Successful!!!")

Train model
Running command: 

   NEURON_RT_ASYNC_EXEC_MAX_INFLIGHT_REQUESTS=3  NEURON_CC_FLAGS='--cache_dir=./compiler_cache --model-type=cnn-training' python -m    yolox.tools.train     -n yolox-s     -d 1     -b 2     


/bin/sh: 1: nvidia-smi: not found
2024-08-26 18:12:24 | INFO     | yolox.core.trainer:142 - args: Namespace(experiment_name='yolox_s', name='yolox-s', dist_backend='nccl', dist_url=None, batch_size=2, devices=1, exp_file=None, resume=False, ckpt=None, start_epoch=None, num_machines=1, machine_rank=0, fp16=False, cache=None, occupy=False, logger='tensorboard', opts=[])
2024-08-26 18:12:24 | INFO     | yolox.core.trainer:143 - exp value:
╒═══════════════════╤════════════════════════════╕
│ keys              │ values                     │
╞═══════════════════╪════════════════════════════╡
│ seed              │ None                       │
├───────────────────┼────────────────────────────┤
│ output_dir        │ './YOLOX_outputs'          │
├───────────────────┼────────────────────────────┤
│ print_interval    │ 10                         │
├───────────────────┼────────────────────────────┤
│ eval_interval     │ 10                         │
├───────────────────┼────────────────────────────┤

Fine-tune Successful!!!
CPU times: user 8.42 ms, sys: 0 ns, total: 8.42 ms
Wall time: 51.7 s


In [9]:
import torch_xla.distributed.xla_backend
import torch_xla
import torch_xla.debug.metrics as met
import torch_xla.distributed.parallel_loader as pl
import torch_xla.utils.utils as xu
import torch_xla.core.xla_model as xm
import torch_xla.distributed.xla_multiprocessing as xmp
import torch

In [23]:
gt_classes = torch.tensor([41., 50.], device='xla:0')
gt_classes

tensor([41., 50.], device='xla:0')

In [24]:
gt_classes.norm()

2024-08-27 19:52:35.000128:  117179  INFO ||NEURON_CACHE||: Compile cache path: /var/tmp/neuron-compile-cache
2024-08-27 19:52:35.000131:  117179  INFO ||NEURON_CC_WRAPPER||: Call compiler with cmd: neuronx-cc compile --target=trn1 --framework=XLA /tmp/ubuntu/neuroncc_compile_workdir/276695cf-a353-49a6-8285-b48a69839d84/model.MODULE_5829231090852593460+d41d8cd9.hlo_module.pb --output /tmp/ubuntu/neuroncc_compile_workdir/276695cf-a353-49a6-8285-b48a69839d84/model.MODULE_5829231090852593460+d41d8cd9.neff --verbose=35
.
Compiler status PASS


tensor(64.6607, device='xla:0')

In [25]:
gt_classes.max()

tensor(50., device='xla:0')

In [14]:
import torch.nn.functional as F
gt_classes = torch.tensor([ 0.,  0., 14., 56., 60.])
num_classes = 80

In [19]:
gt_cls_per_image = (
    F.one_hot((gt_classes).to(torch.int64), num_classes)
    .float()
)

SyntaxError: invalid syntax (582223232.py, line 1)